# Introduction

Python의 Model I/O 에 대해 학습함

FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [ ]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

In [ ]:
chain = prompt | chat

chain.invoke({"country" : "Spain"})

# FewShotChatMessagePromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [ ]:
example_prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        ("human", "what do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a geography expert, you give short answers."),
        example_prompt,
        ("human", "what do you know abiut {country}")
    ]
)

In [ ]:
chain: any= final_prompt | chat
chain.invoke(
    {
        "country", "korea"
    }
)

# RandomBasedExampleSelector

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [ ]:
class RandomExampleSelecter(BaseExampleSelector):
    def __init__(self, examples):
        self.examples: Any = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [ ]:
example_prompt = PromptTemplate = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_sele = RandomExampleSelecter(
    examples=examples
)

In [ ]:
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {counrty}?",
    input_variables=["country"],
)
prompt.format(counrty="Brazil")

# Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

In [ ]:
intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersonating a {character}
  """
)

example = PromptTemplate.from_template(
  """
  this is an example of how you talk:

  Human: {example_question}
  You: {example_answer}
  """
)

start= PromptTemplate.from_template(
  """
  Strat now!

  Human: {question}
  You:
  """
)

final = PromptTemplate.from_template(
  """
  {intro}

  {example}

  {start}
  """
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final,
  pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

# Serialization and Composition

In [32]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

prompt = load_prompt("./prompt.json")


# prompt.format(country="Germmany")

In [6]:
chain = prompt | chat

chain.invoke({
    "country" : "Germmany"
})

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKg6F1zUX4gx831W8cJf2aDgZQ4u3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--305f16fe-12f4-429d-9658-949d50c5cf7c-0', usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# LengthBasedExampleSelector

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

/var/folders/h7/4cv0q0kj0bq39hdg5q154dbw0000gn/T/ipykernel_66685/1671577656.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [13]:
example_prompt: PromptTemplate = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80
)

In [18]:
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
# prompt.format(counrty="Brazil")

In [19]:
chain = prompt | chat

chain.invoke({"country": "spain"})

Spain is a country located in southwestern Europe on the Iberian Peninsula. Its capital city is Madrid. The official language is Spanish (Castilian), and the country is known for its rich cultural heritage, including flamenco music and dance, festivals like La Tomatina and Running of the Bulls, and historic sites such as the Alhambra in Granada and Sagrada Família in Barcelona. Spain's cuisine features dishes like paella, tapas, and churros. The currency used is the Euro (€). Spain is also famous for its diverse landscapes, from beaches along the Mediterranean and Atlantic coasts to mountainous regions like the Pyrenees and Sierra Nevada.

AIMessage(content="Spain is a country located in southwestern Europe on the Iberian Peninsula. Its capital city is Madrid. The official language is Spanish (Castilian), and the country is known for its rich cultural heritage, including flamenco music and dance, festivals like La Tomatina and Running of the Bulls, and historic sites such as the Alhambra in Granada and Sagrada Família in Barcelona. Spain's cuisine features dishes like paella, tapas, and churros. The currency used is the Euro (€). Spain is also famous for its diverse landscapes, from beaches along the Mediterranean and Atlantic coasts to mountainous regions like the Pyrenees and Sierra Nevada.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--a9d65192-2983-4b33-89b9-e519924e5523-0')

# Caching

In [24]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

'The capital of Japan is Tokyo.'

In [21]:
chat.predict("What is the capital of Japan")

'The capital of Japan is Tokyo.'

# Serialization

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug
from typing import Any

set_debug(False)

chat: Any = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

with get_openai_callback() as usage:
    a: Any = chat.predict("What is a Recipt for KFC")
    b: Any = chat.predict("What is a Recipt for Sushi")
    
    print(usage)
    print(usage.total_cost) # 전체 비용
    print(usage.total_tokens) # 토큰 총합
    print(usage.prompt_tokens) # 프롬프트에 사용된 토큰수
    print(usage.completion_tokens) # 모델이 쓴 토큰

Tokens Used: 1268
	Prompt Tokens: 29
		Prompt Tokens Cached: 0
	Completion Tokens: 1239
		Reasoning Tokens: 0
Successful Requests: 2
Total Cost (USD): $0.0007477499999999999
0.0007477499999999999
1268
29
1239
